In [7]:

import create_population

import preprocessing_nar
from feature_engineering import FeatureCreator



import pandas as pd





from train_lightgbm_time import Trainer_lightgbm_time
from train_lgbm_time_cross import Trainer_lightgbm_time_cv

from evaluation_lightgbm_time_kaiki import Evaluator_lightgbm_time_kaiki

from evaluation_shaft_time_kaiki_cross import Evaluator_lightgbm_time_kaiki_shaft

import json
from pathlib import Path


import re

import pandas as pd
import numpy as np
import ast

from pathlib import Path

COMMON_DATA_DIR = Path("..", "..","..",  "common_nar", "data_nar")
POPULATION_DIR_NEW = COMMON_DATA_DIR / "prediction_population"
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
%autoreload

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [10]:
import asyncio
import re
from playwright.async_api import Playwright, async_playwright, expect
import pandas as pd
from io import StringIO

In [41]:
!playwright codegen --target python-async https://www.keiba.go.jp/KeibaWeb/TodayRaceInfo/TodayRaceInfoTop

In [49]:
import asyncio
import re
from playwright.async_api import Playwright, async_playwright, expect
race_id="202543031301" 
place_mapping = {
        1: '札幌',
        2: '函館',
        3: '福島',
        4: '新潟',
        5: '東京',
        6: '中山',
        7: '中京',
        8: '京都',
        9: '阪神',
        10: '小倉',
        30: "門別",
        35: "盛岡",
        36: "水沢",
        42: "浦和",
        43: "船橋",
        44: "大井",
        45: "川崎",
        46: "金沢",
        47: "笠松",
        48: "名古屋",
        50: "園田",
        51: "姫路",
        54: "高知",
        55: "佐賀"

    }
async def run(playwright: Playwright) -> None:
    place_count = f"{int(race_id[10:12])}"
    place_name = f"{place_mapping[int(race_id[4:6])]}"
    browser = await playwright.chromium.launch(headless=False)
    context = await browser.new_context()
    page = await context.new_page()
    await page.goto("https://www.keiba.go.jp/KeibaWeb/TodayRaceInfo/TodayRaceInfoTop")
    await page.get_by_role("link", name=place_name).nth(2).click()
    async with page.expect_navigation():
        # await page.get_by_role("row", name="1R 14:15 Ｃ３三 左1200m 晴 重 12").get_by_role("link").nth(1).click()
        await page.locator("tr").filter(has_text=re.compile(fr"\b{place_count}R\b")).locator("a").nth(1).click()
        # ページ内のリンクテキストをすべて取得して、何があるのか確認する
    html = await page.content()
    html = StringIO(html)
    odds = (
        pd.read_html(html)[0][["馬番","単勝 オッズ"]]
        .set_index("馬番")["単勝 オッズ"]
        .to_dict()
    )
    # ---------------------
    await context.close()
    await browser.close()
    return odds

# async def main() -> None:
async with async_playwright() as playwright:
    odds = await run(playwright)


# asyncio.run(main())


In [50]:
odds

{1: 408.7,
 2: 461.8,
 3: 1.3,
 4: 223.6,
 5: 5.3,
 6: 309.2,
 7: 33.2,
 8: 12.4,
 9: 46.1,
 10: 162.3,
 11: 8.6,
 12: 13.8}

In [28]:
import asyncio
import re
from playwright.async_api import Playwright, async_playwright, expect

playwright = await async_playwright().start()

In [29]:
browser = await playwright.chromium.launch(headless=False)

In [30]:
context = await browser.new_context()

In [31]:
page = await context.new_page()
await page.goto("https://www.keiba.go.jp/KeibaWeb/TodayRaceInfo/TodayRaceInfoTop")

<Response url='https://www.keiba.go.jp/KeibaWeb/TodayRaceInfo/TodayRaceInfoTop' request=<Request url='https://www.keiba.go.jp/KeibaWeb/TodayRaceInfo/TodayRaceInfoTop' method='GET'>>

In [32]:
await page.get_by_role("link", name="船橋").nth(2).click()

In [33]:
await page.get_by_role("row", name="1R 14:15 Ｃ３三 左1200m 晴 重 12").get_by_role("link").nth(1).click()

In [20]:
html = await page.content()
html = StringIO(html)

In [22]:
pd.read_html(html)[0]

,枠,馬番,馬名,単勝 オッズ,複勝オッズ (3着払い),複勝オッズ (3着払い).1,性齢,馬体重 (増減),負担 重量,騎手 (所属),所属,調教師,変更情報
0,1,1,リスクテイカー,408.7,16.2-,35.1,牝5,463（-4）,54.0,藤江渉（川崎）,川崎,村田六,NaN
1,2,2,ボタンブーケ,461.8,31.5-,68.8,牝4,406（+1）,54.0,篠谷葵（船橋）,船橋,石井勝,NaN
2,3,3,ダイユウファリーナ,1.3,1.1-,1.4,牡4,497（+13）,56.0,本田重（船橋）,船橋,新井清,NaN
3,4,4,フォンテーヌパーク,223.6,10.7-,23.1,牡5,451（+9）,53.0,▲山本大（船橋）,船橋,玉井昇,NaN
4,5,5,ハーピーブロッサム,5.3,1.1-,1.6,牝6,478（+5）,54.0,西村栄（船橋）,船橋,山下之,NaN
5,5,6,マイキーマックイン,309.2,29.6-,64.7,セン4,425（+3）,56.0,仲野光（船橋）,船橋,矢内博,NaN
6,6,7,ヴァイナル,33.2,2.2-,4.2,牡4,482（-4）,56.0,木間龍（船橋）,船橋,矢野義,NaN
7,6,8,コスモヴェレッド,12.4,1.6-,3.0,牡6,477（+5）,57.0,川島正（船橋）,船橋,佐々清,NaN
8,7,9,ユウユウロケット,46.1,3.0-,6.1,牡5,455（-6）,56.0,張田昂（船橋）,船橋,渋谷博,NaN
9,7,10,フォースバイオ,162.3,39.0-,85.3,牡5,450（-3）,56.0,山中悠（船橋）,船橋,玉井昇,NaN


In [51]:
    await context.close()
    await browser.close()

In [22]:
COMMON_DATA_DIR = Path("..", "..", "common", "data")
RAWDF_DIR = COMMON_DATA_DIR / "rawdf"
MAPPING_DIR = COMMON_DATA_DIR / "mapping"
POPULATION_DIR = Path("..", "data", "00_population")
OUTPUT_DIR = Path("..", "data", "01_preprocessed")
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)


POPULATION_DIR_NEW = COMMON_DATA_DIR / "prediction_population"

# カテゴリ変数を数値に変換するためのマッピング
with open(MAPPING_DIR / "sex.json", "r") as f:
    sex_mapping = json.load(f)
with open(MAPPING_DIR / "race_type.json", "r") as f:
    race_type_mapping = json.load(f)
with open(MAPPING_DIR / "around.json", "r") as f:
    around_mapping = json.load(f)
with open(MAPPING_DIR / "weather.json", "r") as f:
    weather_mapping = json.load(f)
with open(MAPPING_DIR / "ground_state.json", "r") as f:
    ground_state_mapping = json.load(f)
with open(MAPPING_DIR / "race_class.json", "r") as f:
    race_class_mapping = json.load(f)
with open(MAPPING_DIR / "place.json", "r") as f:
    place_mapping = json.load(f)
populaton_filename = "population_turf.csv"
    
population_dir = POPULATION_DIR
input_dir = RAWDF_DIR
output_dir = OUTPUT_DIR
input_filename = "race_info.csv"
output_filename = "race_info.csv"
race_type_mapping = race_type_mapping
around_mapping = around_mapping
weather_mapping = weather_mapping
ground_state_mapping = ground_state_mapping
race_class_mapping = race_class_mapping

In [23]:
population = pd.read_csv(population_dir / populaton_filename, sep="\t")

# df = pd.read_csv(input_dir / input_filename, sep="\t").query(
#     "race_id in @population['race_id']"
# )
# `race_id`のリストを作成
population_race_ids = population['race_id'].tolist()

# クエリでリストを直接使用
df = pd.read_csv(input_dir / input_filename, sep="\t").query(
    "race_id in @population_race_ids"
)

df


,race_id,title,info1,info2
9,201606010110,ジュニアカップ(OP),"['芝右外1600m', '天候:晴', '芝:良', '発走:14:55', '過去のジュ...","['2016年1月5日', '1回中山1日目', '3歳オープン', '混', '特指', ..."
10,201606010111,第65回日刊スポ賞中山金杯(GIII),"['芝右2000m', '天候:晴', '芝:良', '発走:15:30', '過去の日刊ス...","['2016年1月5日', '1回中山1日目', '4歳以上オープン', '国際', '指'..."
11,201606010112,4歳以上1000万下,"['芝右外1600m', '天候:晴', '芝:良', '発走:16:05']","['2016年1月5日', '1回中山1日目', '4歳以上1000万下', '混', '指..."
21,201608010110,万葉ステークス(OP),"['芝右外3000m', '天候:曇', '芝:良', '発走:15:10', '過去の万葉...","['2016年1月5日', '1回京都1日目', '4歳以上オープン', '国際', 'ハンデ']"
22,201608010111,第54回スポニチ賞京都金杯(GIII),"['芝右外1600m', '天候:曇', '芝:良', '発走:15:45', '過去のスポ...","['2016年1月5日', '1回京都1日目', '4歳以上オープン', '国際', 'ハンデ']"
...,...,...,...,...
31042,202507010910,トリトンステークス(3勝),"['芝左1400m', '天候:晴', '芝:良', '発走:14:50', '過去のトリト...","['2025年1月26日', '1回中京9日目', '4歳以上3勝クラス', '混', '特..."
31044,202507010912,4歳以上2勝クラス,"['芝左1400m', '天候:晴', '芝:良', '発走:16:10']","['2025年1月26日', '1回中京9日目', '4歳以上2勝クラス', '混', '特..."
31053,202510010209,八幡特別(1勝),"['芝右1200m', '天候:晴', '芝:良', '発走:14:05', '過去の八幡特別']","['2025年1月26日', '1回小倉2日目', '4歳以上1勝クラス', '混', '指..."
31054,202510010210,海の中道特別(2勝),"['芝右2600m', '天候:晴', '芝:良', '発走:14:40', '過去の海の中...","['2025年1月26日', '1回小倉2日目', '4歳以上2勝クラス', '混', '特..."


In [24]:
# evalで文字列型の列をリスト型に変換し、一時的な列を作成
df["tmp"] = df["info1"].map(lambda x: eval(x)[0])

# info1 列からコースの長さを取り出して tmp 列を作成
df["tmp2"] = df["info1"].map(lambda x: ast.literal_eval(x))

# tmp 列から距離の部分を抽出
def extract_course_len(info_list):
    for item in info_list:
        match = re.search(r"(\d+)m", item)
        if match:
            return match.group(1)  # マッチした数字部分を返す
    return None  # 該当がなければ None を返す

# コース長を新しい列に追加
df["course_len_1"] = df["tmp2"].apply(extract_course_len)
df["course_len_2"] = df["tmp2"].map(lambda x: x[1]).str.extract(r"(\d+)")
df["combined_course_len"] = df["course_len_1"].fillna(df["course_len_2"])


# ダートor芝or障害
df["race_type"] = df["tmp"].str[0].map(race_type_mapping)

# 右or左or直線
df["around"] = df["tmp"].str[1].map(around_mapping)

# 条件に基づいて course_type を設定
df["course_len_type"] = df["tmp"].apply(
    lambda x: 1 if "内" in str(x[2:3]) else 2 if "外" in str(x[2:3]) else 1
)

df["course_len"] = df["course_len_1"].fillna(df["course_len_2"])
# df["course_len"] = df["tmp"].str.extract(r"(\d+)")
# df["course_len"] = df["tmp"].str.extract(r"(\d+)(?=m)")


df["weather"] = df["info1"].str.extract(r"天候:(\w+)")[0].map(weather_mapping)
df["ground_state"] = (
    df["info1"].str.extract(r"(芝|ダート|障害):(\w+)")[1].map(ground_state_mapping)
)
df["date"] = pd.to_datetime(
    df["info2"].map(lambda x: eval(x)[0]), format="%Y年%m月%d日"
)
regex_race_class = "|".join(race_class_mapping)
df["race_class"] = (
    df["title"]
    .str.extract(rf"({regex_race_class})")
    # タイトルからレース階級情報が取れない場合はinfo2から取得
    .fillna(df["info2"].str.extract(rf"({regex_race_class})"))[0]
    .map(race_class_mapping)
)
df["place"] = df["race_id"].astype(str).str[4:6].astype(int)
df.dropna(subset=["place"], inplace=True)

df["course_type"] = df["place"].astype(str) + df["course_len"].astype(str) + df["course_len_type"].astype(str)


# 年、月、日をそれぞれ抽出
df["date_year"] = df["date"].dt.year
df["date_month"] = df["date"].dt.month
df["date_day"] = df["date"].dt.day

In [25]:
df

,race_id,title,info1,info2,tmp,tmp2,course_len_1,course_len_2,combined_course_len,race_type,around,course_type,course_len,weather,ground_state,date,race_class,place,date_year,date_month,date_day
9,201606010110,ジュニアカップ(OP),"['芝右外1600m', '天候:晴', '芝:良', '発走:14:55', '過去のジュ...","['2016年1月5日', '1回中山1日目', '3歳オープン', '混', '特指', ...",芝右外1600m,"[芝右外1600m, 天候:晴, 芝:良, 発走:14:55, 過去のジュニアカップ]",1600,NaN,1600,1,0,2,1600,1,0,2016-01-05,5,6,2016,1,5
10,201606010111,第65回日刊スポ賞中山金杯(GIII),"['芝右2000m', '天候:晴', '芝:良', '発走:15:30', '過去の日刊ス...","['2016年1月5日', '1回中山1日目', '4歳以上オープン', '国際', '指'...",芝右2000m,"[芝右2000m, 天候:晴, 芝:良, 発走:15:30, 過去の日刊スポ賞中山金杯]",2000,NaN,2000,1,0,0,2000,1,0,2016-01-05,6,6,2016,1,5
11,201606010112,4歳以上1000万下,"['芝右外1600m', '天候:晴', '芝:良', '発走:16:05']","['2016年1月5日', '1回中山1日目', '4歳以上1000万下', '混', '指...",芝右外1600m,"[芝右外1600m, 天候:晴, 芝:良, 発走:16:05]",1600,NaN,1600,1,0,2,1600,1,0,2016-01-05,3,6,2016,1,5
21,201608010110,万葉ステークス(OP),"['芝右外3000m', '天候:曇', '芝:良', '発走:15:10', '過去の万葉...","['2016年1月5日', '1回京都1日目', '4歳以上オープン', '国際', 'ハンデ']",芝右外3000m,"[芝右外3000m, 天候:曇, 芝:良, 発走:15:10, 過去の万葉ステークス]",3000,NaN,3000,1,0,2,3000,2,0,2016-01-05,5,8,2016,1,5
22,201608010111,第54回スポニチ賞京都金杯(GIII),"['芝右外1600m', '天候:曇', '芝:良', '発走:15:45', '過去のスポ...","['2016年1月5日', '1回京都1日目', '4歳以上オープン', '国際', 'ハンデ']",芝右外1600m,"[芝右外1600m, 天候:曇, 芝:良, 発走:15:45, 過去のスポニチ賞京都金杯]",1600,NaN,1600,1,0,2,1600,2,0,2016-01-05,6,8,2016,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31042,202507010910,トリトンステークス(3勝),"['芝左1400m', '天候:晴', '芝:良', '発走:14:50', '過去のトリト...","['2025年1月26日', '1回中京9日目', '4歳以上3勝クラス', '混', '特...",芝左1400m,"[芝左1400m, 天候:晴, 芝:良, 発走:14:50, 過去のトリトンステークス]",1400,NaN,1400,1,1,0,1400,1,0,2025-01-26,4,7,2025,1,26
31044,202507010912,4歳以上2勝クラス,"['芝左1400m', '天候:晴', '芝:良', '発走:16:10']","['2025年1月26日', '1回中京9日目', '4歳以上2勝クラス', '混', '特...",芝左1400m,"[芝左1400m, 天候:晴, 芝:良, 発走:16:10]",1400,NaN,1400,1,1,0,1400,1,0,2025-01-26,3,7,2025,1,26
31053,202510010209,八幡特別(1勝),"['芝右1200m', '天候:晴', '芝:良', '発走:14:05', '過去の八幡特別']","['2025年1月26日', '1回小倉2日目', '4歳以上1勝クラス', '混', '指...",芝右1200m,"[芝右1200m, 天候:晴, 芝:良, 発走:14:05, 過去の八幡特別]",1200,NaN,1200,1,0,0,1200,1,0,2025-01-26,5,10,2025,1,26
31054,202510010210,海の中道特別(2勝),"['芝右2600m', '天候:晴', '芝:良', '発走:14:40', '過去の海の中...","['2025年1月26日', '1回小倉2日目', '4歳以上2勝クラス', '混', '特...",芝右2600m,"[芝右2600m, 天候:晴, 芝:良, 発走:14:40, 過去の海の中道特別]",2600,NaN,2600,1,0,0,2600,1,0,2025-01-26,5,10,2025,1,26
